In [ ]:
from src.utils import finetune, get_dataset, get_classification_head, get_baseline_encoder
from src.modeling import Classifier
from monai.data import DataLoader
from pathlib import Path
import torch
from src.evaluation import evaluate_segmentation_performance


In [6]:
dataset_names = ["CHAOS"]
domains = ["CT", "MRI"]
save_path = "checkpoints/"

In [7]:
datasets = {(name, domain): get_dataset(name, domain) for name in dataset_names for domain in domains}

Loading dataset: 100%|██████████| 20/20 [00:00<00:00, 350987.78it/s]


In [ ]:
# Baseline evaluation loop

encoder = get_baseline_encoder(save_path)
for dataset_name in ["CHAOS",]:
    for domain in domains:
        filename = Path(save_path) / f"{dataset_name}_{domain}_baseline.pth"
        dataloader = DataLoader(datasets[(dataset_name, domain)], batch_size=1, shuffle=False, num_workers=4)

        # Initialize the classifier
        classification_head = get_classification_head(dataset_name, domain)
        classifier = Classifier(encoder, classification_head)

        dice, hauseldorff = evaluate_segmentation_performance(dataloader, classifier)


In [ ]:
# Finetuning loop

for dataset_name in dataset_names:
    for domain in domains:
        print(f"Finetuning on {dataset_name} dataset in {domain} domain")
        dataset = get_dataset(dataset_name, domain, is_train=True,base_path = "data/")
        loader = DataLoader(dataset, batch_size=32, shuffle=True)
        # encoder = # Base pretrained encoder model
        # head = get_classification_head(dataset_name, domain)
        # model = Classifier(encoder, head)
        # tuned_model = finetune(model, loader, save_path)

Finetuning on CHAOS dataset in CT domain


Loading dataset: 100%|██████████| 20/20 [00:00<00:00, 86302.55it/s]


Finetuning on CHAOS dataset in MRI domain


Loading dataset: 100%|██████████| 20/20 [00:00<00:00, 167772.16it/s]


Finetuning on MM-WHS dataset in CT domain
Extracting data/MM-WHS/MM-WHS 2017 Dataset.zip to data/MM-WHS/MM-WHS 2017 Dataset...
Extracting data/MM-WHS/MMWHS_evaluation_testdata_label_encrypt_1mm_forpublic.zip to data/MM-WHS/MMWHS_evaluation_testdata_label_encrypt_1mm_forpublic...


NotImplementedError: Dataset MM-WHS is not implemented.